In [32]:
from json import loads
import requests
# from transformers import pipeline
# classifier = pipeline("zero-shot-classification",
#                       model="facebook/bart-large-mnli")

apiKey = "hTLGygty2Sj5IB368rcArA63Xu29hW2r"
archiveUrl = f'https://api.nytimes.com/svc/archive/v1/#1/#2.json?api-key={apiKey}'




res = requests.get(archiveUrl.replace("#1", "2010").replace("#2", "9"))
obj = json.loads(res.text)


with open("september.json", "w", encoding="utf-8") as json_file:
    json_file.write(res.text)

res = requests.get(archiveUrl.replace("#1", "1969").replace("#2", "10"))
obj = json.loads(res.text)
    
with open("october.json", "w", encoding="utf-8") as json_file:
    json_file.write(res.text)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBartForSequenceClassification: ['model.encoder.version', 'model.decoder.version']
- This IS expected if you are initializing TFBartForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBartForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBartForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBartForSequenceClassification for predictions without further training.


In [33]:
#sequence_to_classify = "California has systemic issues!"
#candidate_labels = ['geopolitics',"sports", "education","USA"]
#classifier(sequence_to_classify, candidate_labels)

{'sequence': 'California has systemic issues!',
 'labels': ['USA', 'geopolitics', 'sports', 'education'],
 'scores': [0.4686087667942047,
  0.2426447719335556,
  0.14566738903522491,
  0.14307904243469238]}

In [35]:
with open("october.json", "r", encoding="UTF-8") as json_file:
    october_json = loads(json_file.read())

with open("september.json", "r", encoding="UTF-8") as json_file:
    september_json = loads(json_file.read())

# list_of_docs = [doc["abstract"] for doc in october_json]
# for index, doc in enumerate(list_of_docs):
#     if doc != "":
#         classification = classifier(doc, candidate_labels)
#         print(classification)
#         print(doc)
#         if (index+1)%100 == 0:
#             break

If "news_desk" == "" choose section_name
    If section_name == "Archive"
        => Zero Shot
If ("news_desk" == "None" || "news_desk" == "Letters")
        => Zero Shot

In [38]:
def get_classification_labels(doc_json: dict):
    topics = []
    for doc in doc_json:
        topic = doc["news_desk"]
        if doc["news_desk"] == "None" or doc["news_desk"] == "":
            if  doc["section_name"] != "Archives":
                topic = doc["section_name"]
        if topic not in topics:
            topics.append(topic)
    return topics
        


In [46]:
groups = {}
print(len(october_json))

candidate_labels = get_classification_labels(september_json["response"]["docs"])
print(candidate_labels)

def group_by_topic(doc_json: dict):

    for index, doc in enumerate(doc_json):
        topic = doc["news_desk"]
        if doc["news_desk"] == "None" or doc["news_desk"] == "":
            topic = doc["section_name"]
            if topic == "Archives":
                if doc["abstract"] != "":
                    #print(doc["abstract"])
                    #print(classification)
                    topic = "undefined"#classifier(doc["abstract"], candidate_labels[0:4])["labels"][0]
                    #print(topic)
                else:
                    continue

        if topic in groups.keys():
            groups[topic].append(doc["abstract"])
        else:
            groups[topic] = [doc["abstract"]]
        # if index%1== 0:
        #     print(index)
        #     break
    return groups

groups = group_by_topic(september_json["response"]["docs"])

groups


11585
['Foreign', 'Sports', 'Business', 'Metro', 'National', 'Letters', 'OpEd', 'Editorial', 'Summary', 'Travel', 'U.S.', 'Health', 'Education', 'Business Day', 'Opinion', 'Multimedia/Photos', 'Week in Review', 'Technology', 'Automobiles', 'Culture', 'Magazine', 'TStyle', 'Arts', 'Blogs', 'T Magazine', 'Your Money', 'Style', 'Styles', 'Media', 'Science', 'ContinuousNews', '0', 'Movies', 'Fashion & Style', 'World', 'Home & Garden', 'Home', 'Great Homes & Destinations', 'Crosswords & Games', 'Weekend', 'The Learning Network', 'RealEstate', 'New York', 'Real Estate', 'BookReview', 'Society', 'Books', 'Dining', 'SundayBusiness', 'WeekInReview', 'Circuits', 'Times Topics', 'Food', 'Theater', 'Washington', 'Obituaries']


{'Foreign': ['The text, as prepared for delivery, of President Obama’s address from the Oval Office on Tuesday night, provided by the White House.',
  'An Iraqi proverb resonates today.',
  'Many of the same complaints about daily life are heard now as in 2003, when the United States invasion transformed the political life of the nation.',
  'President Obama is looking for simultaneous progress on Iraq, Iran and Israeli-Palestinian peace, a triple play that has eluded his predecessors for decades.',
  'The summer of 2006 felt like a fragile turning point, a tenuous peace that could easily be interrupted. The country today is at another turning point. And its peace may be no less fragile.',
  'The organization and leadership of the miners will be essential to their rescue, officials said.',
  'Eight employees of a Cancun strip club were killed early Tuesday when a group of men threw Molotov cocktails into the building in a working-class area not frequented by tourists.',
  'Riot police 

In [58]:
from sklearn.feature_extraction.text import TfidfVectorizer
#Get words that are in at least 1% of documents and at most 5% of documents
vectorizer = TfidfVectorizer(min_df = 0.01, max_df = 0.05, stop_words="english")

X = vectorizer.fit_transform(groups["Foreign"])
a = vectorizer.get_feature_names_out()

print(a)
print(len(a))

['000' '10' 'abuse' 'accused' 'administration' 'afghan' 'afghanistan' 'al'
 'announced' 'attack' 'attacks' 'authorities' 'baghdad' 'bank' 'beijing'
 'britain' 'british' 'capital' 'captain' 'charges' 'children' 'china'
 'chinese' 'church' 'city' 'combat' 'construction' 'countries' 'country'
 'crisis' 'david' 'day' 'days' 'dead' 'deal' 'decision' 'defense'
 'despite' 'did' 'died' 'early' 'election' 'elections' 'end' 'european'
 'face' 'family' 'forces' 'foreign' 'france' 'french' 'friday' 'gates'
 'gaza' 'germany' 'group' 'groups' 'held' 'help' 'helped' 'high' 'house'
 'hundreds' 'immigration' 'including' 'india' 'involved' 'iran' 'iranian'
 'iraq' 'iraqi' 'israel' 'israeli' 'japan' 'kabul' 'killed' 'killing'
 'kim' 'korea' 'law' 'leader' 'leaders' 'led' 'left' 'life' 'long'
 'mahmoud' 'members' 'men' 'million' 'monday' 'money' 'month' 'months'
 'mr' 'nation' 'national' 'nations' 'near' 'negotiations' 'new' 'news'
 'newspaper' 'north' 'northern' 'nuclear' 'number' 'obama' 'official'
 'op